**Generative networks (part 1)- Generating pictures of handwritten numbers from known pictures**

A generative network's job is, rather than classifying input, to generate output based on some output. This means that there's no real answer for such a network to compare its output to. One such way of solving this problem, however,is to give the network a picture of a known number and let it run its weights and activations on that image, and compare the result to the original image. This set of networks does that. The first takes known images and encodes them, then decodes them to create its own images. The second takes the images that the first one makes and classifies them to verify that they are good. 

Example:

Let's say we have an image that has 9 pixels, in a 3x3 arrangement (the numbers are the greyscale illumination of the pixels):

[0.5][0.4][0.2]      
[0.7][0.3][0.9]       
[0.3][0.2][0.1] 

The network then runs this image through itself and this is the result:

[0.3][0.6][0.2]      
[0.5][0.7][0.1]     
[0.9][0.4][0.0]

This is compared back to the original image so network performance can be measured (i.e how close to the original image this really is).

The goal of this assignment is for the neural network to geenrate pictures of digits. I have tried 2 approaches to for this, and the one I just described is the first. 

While this does work fairly well for generating images, technically it doesn't count since an image of the number is required for the network to make its own image. So we want the network to be able to generate an image from random pixels. This is explored in part 2 of this assignment. 

In [0]:
'''
November 7th, 2019

Abhi Jain

Mr.Saxberg

This is the generator portion of the neural network. It looks at an image of a known digit and tries its best to 
recreate it, then compares its rendition back to the original.
'''

#import libraries
#machine learning libraries
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets
#other libraries
import numpy as np
import random
import collections
import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = [10, 8] #changing image plotting parmeters (for display only)

(train_images, train_labels), (test_images, test_labels) = datasets.mnist.load_data() #getting train and test data for digits

train_images = train_images.reshape((train_images.shape[0], 784)).astype('float32') #reshaping train data
train_images = train_images / 255 #converting pixel values to be between 0 and 1

test_images = test_images.reshape((test_images.shape[0], 784)).astype('float32') #reshaping test data
test_images = test_images / 255 #converting pixel values to be between 0 and 1

print(train_images[61].shape)

#neural network
model = keras.Sequential([
    keras.layers.Dense(30,input_dim=784, activation=tf.nn.relu), #input layer- distributes 784 pixel values between 30 neurons (encoding)
    keras.layers.Dense(784, activation=tf.nn.relu) #output layer- decodes the encoder values and recreates a new picture
])

model.compile(optimizer='adam',  
              loss='mean_squared_error', #most direct comparison between image and generated image
              metrics=["mean_squared_error", "accuracy"])

model.fit(train_images, train_images, epochs=100, batch_size=200) #training network


test_loss, test_acc, _ = model.evaluate(test_images, test_images) #evaluating performance
print(test_acc)

In [0]:
'''
This portion of the code gives te generator network test images for it to generate images, which are then stored in a prediction database
'''

prediction_database = []

print(test_images[61].shape)

for i in range(len(test_images)):
    prediction_database.append(model.predict(np.array([test_images[i]], dtype="float"))) #getting prediction images

prediction_database = np.array(prediction_database, dtype="float")
print(np.array(prediction_database, dtype="float").shape)

In [0]:
'''
This portion of code displays an image the network created
'''

plt.imshow(np.reshape(prediction_database[0], (28,28)))

In [0]:
'''
This portion of the code is a simple image classifier. It takes the images the generator made and then classifies them. Note that we know what each image
is supposed to be since they are based on test images, which all have labels 
'''

#classifier network
model2 = keras.Sequential([
    keras.layers.Dense(784, activation = tf.nn.tanh), #input layer
    keras.layers.Dense(50, activation=tf.nn.tanh), #hidden layer
    keras.layers.Dense(30, activation=tf.nn.tanh), #hidden layer
    keras.layers.Dense(10, activation=tf.nn.softmax) #output layer with 10 neurons, one for each possible answer (digits from 0-9)
])

model2.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=["accuracy"])

model2.fit(train_images, train_labels, epochs=6, batch_size=200) #training it on known test images so it can classify well

test_loss, test_acc = model2.evaluate(prediction_database.reshape((10000,784)), test_labels) #evaluating performance

predictions = model2.predict(prediction_database.reshape((10000,784))) #predicting what the generated images are

#comparing the prediction of the first 15 images to their actual labels 
for i in range(15):
    print(np.round(predictions[i]*1000)/1000, np.argmax(predictions[i]), test_labels[i])
